# **MACHINE LEARNING**


**1.** Apresentação de um problema real que pode ser resolvido com
modelo supervisionado (2~3 slides contendo o tipo de problema e a
técnica/modelo que resolveria);

    Resposta : A possibilidade de evasão de um aluno de um curso de graduação. 
    Modelo   : Classificação (Regressão  Logística).

**2.** Apresentação de um problema real que pode ser resolvido com
modelo não-supervisionado (2~3 slides contendo o tipo de problema
e a técnica/modelo que resolveria);

    Resposta : Segmentação de carreira de alunos com base em performance em cada área de atuação escolar. 
    Modelo   : Clustering (K-Means).

**3.** Apresentação de um problema real que pode ser resolvido usando ML,
mas não precisa de fato;

    Resposta : Detectar possível hacker em uma rede interna com base no seu comportamento. O mesmo pode ser verificado com base em IPs ou portas acessadas pelo usuário, colocando as condições de identificação ou bloqueio via Firewall.
    Modelo   : Classificação (Regressão  Logística) ou Deep Learning.

**4.** Apresentar um problema implementado (com código), que pode ser:

    a. Um artigo científico que usou algum modelo visto na disciplina
    para resolver um determinado problema. 
    b. Um problema de alguma competição (e.g. Kaggle) que usou
    algum modelo visto na disciplina.
    c. Escolher qualquer problema que o grupo queira (da empresa
    que trabalha, StartupOne) e criar uma aplicação de ML. 

## Modelo de Previsão de Partidas da Copa do Mundo

O objetivo do modelo é prever o resultado de jogos da Copa do Mundo de futebol de 2022 no Quatar.
O intuito do projeto é utilizar os modelos supervisionados de classificação para obter o resultado esperado de cada partida da competição, chegando no possível ganhador.

In [1]:
# Carregamos as principais bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
# Realizamos a leitura do dataset
dados = pd.read_csv("results.csv")

In [3]:
# Fazemos a descrição dos dados:
dados.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [4]:
# verificamos os tipos de dados
# verificamos se temos dados vazios que precisam ser tratados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [5]:
# Removemos os NAs presentes
dados = dados.dropna();

# Covertemos os placares para inteiros
dados["home_score"] = dados["home_score"].astype("int")
dados["away_score"] = dados["away_score"].astype("int")

dados[["home_score","away_score"]]

,home_score,away_score
0,0,0
1,4,2
2,2,1
3,2,2
4,3,0
...,...,...
44054,1,1
44055,0,2
44056,1,1
44057,5,1


In [6]:
# Separamos os valores de data em colunas diferentes
dados[["year", "month", "day"]] = dados["date"].str.split("-", expand = True)

# Convertemos as strings para inteiros
dados[["year", "month", "day"]] = dados[["year", "month", "day"]].astype("int")
dados[["year", "month", "day"]]

dados.drop("date", axis=1)

,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day
0,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11,30
1,England,Scotland,4,2,Friendly,London,England,False,1873,3,8
2,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,3,7
3,England,Scotland,2,2,Friendly,London,England,False,1875,3,6
4,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,3,4
...,...,...,...,...,...,...,...,...,...,...,...
44054,Albania,Iceland,1,1,UEFA Nations League,Tirana,Albania,False,2022,9,27
44055,Norway,Serbia,0,2,UEFA Nations League,Oslo,Norway,False,2022,9,27
44056,Sweden,Slovenia,1,1,UEFA Nations League,Stockholm,Sweden,False,2022,9,27
44057,Kosovo,Cyprus,5,1,UEFA Nations League,Pristina,Kosovo,False,2022,9,27


In [7]:
# Mudamos os valores de cada coluna categorica para o tipo category, para futuramente transformá-los em números
dados["home_team"]  = dados["home_team"].astype('category')
dados["away_team"]  = dados["away_team"].astype('category')
dados["tournament"] = dados["tournament"].astype('category')
dados["city"]       = dados["city"].astype('category')
dados["country"]    = dados["country"].astype('category')
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44059 entries, 0 to 44058
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   date        44059 non-null  object  
 1   home_team   44059 non-null  category
 2   away_team   44059 non-null  category
 3   home_score  44059 non-null  int32   
 4   away_score  44059 non-null  int32   
 5   tournament  44059 non-null  category
 6   city        44059 non-null  category
 7   country     44059 non-null  category
 8   neutral     44059 non-null  bool    
 9   year        44059 non-null  int32   
 10  month       44059 non-null  int32   
 11  day         44059 non-null  int32   
dtypes: bool(1), category(5), int32(5), object(1)
memory usage: 2.1+ MB


In [8]:
# Convertemos as colunas categoricas para inteiros
dados["home_team"] = dados["home_team"].cat.codes
dados["away_team"] = dados["away_team"].cat.codes
dados["tournament"] = dados["tournament"].cat.codes
dados["city"] = dados["city"].cat.codes
dados["country"] = dados["country"].cat.codes
dados.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day
0,1872-11-30,238,83,0,0,70,642,204,False,1872,11,30
1,1873-03-08,83,233,4,2,70,1003,68,False,1873,3,8
2,1874-03-07,238,83,2,1,70,642,204,False,1874,3,7
3,1875-03-06,83,233,2,2,70,1003,68,False,1875,3,6
4,1876-03-04,238,83,3,0,70,642,204,False,1876,3,4


In [13]:
# Criamos uma coluna para mencionar o vencedor da partida

for x in dados:
    if(dados["home_score"][x] > dados["away_score"][x]):
        dados["winner"][x] = np.append(dados["home_score"][x])
    elif(dados["home_score"][x] < dados["away_score"][x]):
        dados["winner"][x] = np.append(dados["away_score"][x])
    else:
        dados["winner"][x] = np.append(0)

dados.head()

KeyError: 'date'